## Website
https://medium.com/microsoftazure/recommendation-systems-enhanced-by-llms-fe1fc8e23a58

In [1]:
import os
import pandas as pd
import lancedb
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import LanceDB
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAI

from dotenv import load_dotenv
load_dotenv(verbose=True, override=True)

embeddings_api = AzureOpenAIEmbeddings(azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
                                    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")
                                    )

llm_chat_api = AzureChatOpenAI(openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
                            azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
                            max_tokens = 1500
                            )

llm_instruct_api = AzureOpenAI(openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
                            azure_deployment=os.environ["AZURE_OPENAI_INSTRUCT_DEPLOYMENT_NAME"],
                            max_tokens = 1500
                            )



c:\Users\jpimenta\AppData\Local\anaconda3\envs\bmw-001\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
card_rewards = pd.read_pickle(os.path.join(os.getcwd(), "data","card_rewards.pkl"))  
card_rewards.head(2)

,CardName,Discount,Description,text,n_tokens,vector
0,American Express Gold Card,€30,Get a €30 statement credit by spending €150 or...,CardName: American Express Gold Card. Discount...,42,"[-0.028317386284470558, -0.01587647572159767, ..."
1,American Express Gold Card,€25,Earn a €25 statement credit with a €99 purchas...,CardName: American Express Gold Card. Discount...,43,"[-0.021910404786467552, -0.013033478520810604,..."


we want to store our embeddings into a VectorDB, so that we can perform similarity search with the embedded query of the user. LangChain offers many integrations with 3rd party vector stores and, in this case, we are going to use LanceDB:

In [3]:
uri = "dataset/sample-card-lancedb"
db = lancedb.connect(uri)
table = db.create_table("card", card_rewards, mode="overwrite")

embeddings = embeddings_api

docsearch = LanceDB(connection = table, embedding = embeddings)

To test our vector store, let’s start with a simple similarity computation which returns the first most similar result using cosine similarity as distance metric:

In [4]:
query = "I'm looking for an grocery shop credit cad promotion. What card could you suggest to me?"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content='CardName: Mastercard Platinum. Discount: 5% cashback Description: Earn 5% cashback on all grocery store purchases. Offer valid until 11/30/2023.', metadata={'CardName': 'Mastercard Platinum', 'Discount': '5% cashback', 'Description': 'Earn 5% cashback on all grocery store purchases. Offer valid until 11/30/2023.', 'n_tokens': 39, 'vector': array([-0.01746309, -0.01555755, -0.00399054, ...,  0.00378824,
        -0.0067738 , -0.03033201], dtype=float32), '_distance': 0.30936431884765625})]

Use LangChain RetrievalQA chain, alongside the LanceDB as retriever.

In [5]:
# Import Azure OpenAI
query = "I'm looking for an grocery shop credit cad promotion. What card could you suggest to me?"

# qa = RetrievalQA.from_chain_type(llm=llm_chat_api, 
#                                 chain_type="stuff", 
#                                 retriever=docsearch.as_retriever(), 
#                                 return_source_documents=True
#                                 )

# result = qa({"query": query})
# print(f"Chat: {result['result']}" )

qa = RetrievalQA.from_chain_type(llm=llm_instruct_api, 
                                chain_type="stuff", 
                                retriever=docsearch.as_retriever(), 
                                return_source_documents=True
                                )

result = qa({"query": query})
print(f"Result: {result['result']}" )

c:\Users\jpimenta\AppData\Local\anaconda3\envs\bmw-001\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Chat: Based on the information provided, the Mastercard Platinum would be a great choice for grocery shopping as it offers 5% cashback on all grocery store purchases, and this offer is valid until 11/30/2023. Another option is the Capital One SavorOne Cash Rewards Credit Card, which offers unlimited 3% cash back at grocery stores among other categories. If you are looking for a card with a limited time offer, the Revolut Standard Card also provides 2% cashback on grocery store purchases.
Instruct:  The Capital One SavorOne Cash Rewards Credit Card or the Capital One Walmart Rewards Card would be good options since they both offer 5% cashback on grocery store purchases. However, the Revolut Standard Card also offers 2% cashback on all grocery store purchases for a limited time.


Since we used the return_source_documents=True parameter, we can also retrieve the source documents from our result variable:

In [6]:
result['source_documents'][0]

Document(page_content='CardName: Mastercard Platinum. Discount: 5% cashback Description: Earn 5% cashback on all grocery store purchases. Offer valid until 11/30/2023.', metadata={'CardName': 'Mastercard Platinum', 'Discount': '5% cashback', 'Description': 'Earn 5% cashback on all grocery store purchases. Offer valid until 11/30/2023.', 'n_tokens': 39, 'vector': array([-0.01746309, -0.01555755, -0.00399054, ...,  0.00378824,
       -0.0067738 , -0.03033201], dtype=float32), '_distance': 0.30936431884765625})

With RetrievalQA, you can pass a custom prompt that you can easily define using LangChain prompt templates. Let’s start with a simple prompt as follows:

In [7]:
from langchain.prompts import PromptTemplate

template = """You are a credit card recommender system that help me to decide which card I should use in my next payment. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three cards, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

query = "I'm looking for an grocery shop credit cad promotion. What card could you suggest to me?"

qa = RetrievalQA.from_chain_type(llm=llm_instruct_api, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


result = qa({'query':query})

print(f"Result: {result['result']}" )


Instruct:  Three cards that could be suitable for you are the Mastercard Platinum, Capital One SavorOne Cash Rewards Credit Card, and Capital One Walmart Rewards Card.
The Mastercard Platinum offers 5% cashback on all grocery store purchases, making it a great choice for someone who shops at grocery stores frequently. The Capital One SavorOne Cash Rewards Credit Card also offers 3% cashback on grocery store purchases, along with additional benefits for dining and entertainment. Lastly, the Capital One Walmart Rewards Card offers 5% cashback on Walmart.com purchases, making it a great option for someone who frequently shops at Walmart for groceries.


In [26]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a credit card recommender system that help me to decide which card the user should use in my next payment. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
Take special attetion to the store were the user currently is and only suggest him/her to use the credit card that has the most benefits for the specific store.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}
PreviousPayments: {listPreviousPayments}
Store:{store}
"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, 
                            gender = 'female', 
                            listPreviousPayments="['SPORTING GOODS STORES', 'COMPUTERS, COMPUTER PERIPHERAL EQUIPMENT, SOFTWARE', 'TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES', 'BOOK STORES']",
                            store='Mcdonals'
)

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a credit card recommender system that help me to decide which card the user should use in my next payment. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
Take special attetion to the store were the user currently is and only suggest him/her to use the credit card that has the most benefits for the specific store.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: female
PreviousPayments: ['SPORTING GOODS STORES', 'COMPUTERS, COMPUTER PERIPHERAL EQUIPMENT, SOFTWARE', 'TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES', 'BOOK STORES']
Store:Mcdonals

Question: {question}
Your response:


In [27]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

query = "Can you suggest me which credit card should I use?"

qa = RetrievalQA.from_chain_type(llm=llm_instruct_api, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


result = qa({'query':query})
result['result']


print(f"Result: {result['result']}" )

Result:  Based on the information provided, I would recommend using the Capital One SavorOne Cash Rewards Credit Card. This card offers 3% cash back on dining, which would be beneficial for your current purchase at McDonald's. Additionally, the card offers cash back on entertainment, streaming services, and grocery stores, which align with your previous payments at sporting goods stores, computer stores, and book stores. This card also has no annual fee and offers a higher credit line after making your first 5 monthly payments on time, making it a good choice for building credit. 
